# Проверка различных значений

При создании выпукло-двойственной задачи мы дважды используем значения $x_0, z_0$. Один раз они являются ограничениями правой части, другой - коэффициентами.

При этом нам бы хотелось иметь возможность менять ограничеия правой части и при этом оставлять неизменными коэффициенты. Тогда получится встраивать такие ограничения в симплекс метод.

Попробуем заменить в ограничениях, которое отображает равенство целевых функций $x$ на $x_0 + \Delta x$ и $l$ на $l_0 + \Delta l$

In [1]:
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
xa,xb = 0,1
za,zb = 0,1
grid_step = 0.1
xa_ext = xa - grid_step
xb_ext = xb + grid_step
za_ext = za - grid_step
zb_ext = zb + grid_step


x = np.arange(xa_ext, xb_ext, grid_step, dtype=np.double)
z = np.arange(za_ext, zb_ext, grid_step, dtype=np.double)
x,z = np.meshgrid(x, z)
y_cvx = x**2 + x * z + z**2
y_ccv = - x**2 - z**2

x = x.flatten()
z = z.flatten()
y_cvx = y_cvx.flatten()
y_ccv = y_ccv.flatten()

mask = x+z <= 1+grid_step

xs = x[mask]
zs = z[mask]
ys_pos = y_cvx[mask]
ys_neg = y_ccv[mask]

Для начала узнаем в как варьируется $l$

In [28]:
def f(xs, zs, y_pos, y_neg, x, z, l_01, l_02, bx, bz):
    dim_p = len(xs)
    rhs_p = np.array([x,z, 1], dtype=np.double)
    dim_d = len(rhs_p) 

    s_pos = CyClpSimplex()
    u_pos = s_pos.addVariable('u', dim_p)
    l_pos = s_pos.addVariable('l', dim_d)

    s_neg = CyClpSimplex()
    u_neg = s_neg.addVariable('u', dim_p)
    l_neg = s_neg.addVariable('l', dim_d)


    A_p_pos = np.vstack([y_pos,
                         xs,
                         zs,
                         np.ones(dim_p)])
    A_p_neg = np.vstack([y_neg,
                         xs,
                         zs,
                         np.ones(dim_p)])

    A_p_pos = np.matrix(A_p_pos)
    A_p_neg = np.matrix(A_p_neg)

    b_p = CyLPArray(np.hstack([0,rhs_p]))
    b_d_pos = CyLPArray(y_pos)
    b_d_neg = CyLPArray(y_neg)

    A_d = np.hstack([xs.reshape(-1, 1),
                     zs.reshape(-1, 1),
                     np.ones(len(xs)).reshape(-1, 1)])
    A_d = np.matrix(A_d)


    # A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
    t = np.array([bx, bz, 1], dtype=np.double)
    A_d1 = np.matrix(np.vstack([t,np.zeros((3,3))]))

#     dx = bx - x_0
#     dz = bz - z_0
    t = np.array([x, z, 0], dtype=np.double)
    A_d2 = np.matrix(np.vstack([t,np.zeros((3,3))]))

    #l_0 = np.array([1, 1, 1], dtype=np.double)
    
    s_pos += A_p_pos*u_pos - np.dot(A_d1,l_01) + A_d1*l_pos + np.dot(A_d2,l_01) == b_p
    s_neg += A_p_neg*u_neg - np.dot(A_d1,l_02) + A_d1*l_neg + np.dot(A_d2,l_02) == b_p

    for i in range(dim_p):
        s_pos += u_pos[i] >= 0
        s_neg += u_neg[i] >= 0

    s_pos += A_d*l_pos <= b_d_pos
    s_neg += A_d*l_neg >= b_d_neg

    s_pos.objective = u_pos[0]
    s_neg.objective = u_neg[0]

    s_pos.primal()
    s_neg.primal()

    cond_pos = s_pos.primalVariableSolution['u']
    yr_pos = np.dot(cond_pos, y_pos)

    cond_neg = s_neg.primalVariableSolution['u']
    yr_neg = np.dot(cond_neg, y_neg)
    
    xr = np.dot(cond_pos, xs)
    zr = np.dot(cond_pos, zs)
    return yr_pos + yr_neg, x*z, s_pos.getStatusString(), s_neg.getStatusString(),xr,zr, s_pos.primalVariableSolution['l'], s_neg.primalVariableSolution['l']

In [29]:
r = f(xs,zs,ys_pos,ys_neg,0.3,0.4,np.array([0,0,0]),np.array([0,0,0]),0.5,0.5)
r

(0.11000000000000004,
 0.12,
 'optimal',
 'optimal',
 0.3,
 0.4,
 array([ 0.        ,  0.        , -0.42470588]),
 array([0.        , 0.        , 0.31470588]))

In [27]:
r = f(xs,zs,ys_pos,ys_neg,0.3,0.4,np.array([0. ,  0. , -0.42470588]),np.array([0., 0., 0.31470588]),0.3,0.4)
r

(0.11000000000000004,
 0.12,
 'optimal',
 'optimal',
 0.3,
 0.4,
 array([ 0.        ,  0.        , -0.42470588]),
 array([0.        , 0.        , 0.31470588]))

In [12]:
r = f(xs,zs,ys_pos,ys_neg,0.5,0.5,np.array([ 1.5,  1.4, -0.7]),np.array([-0.9, -0.9,  0.4]),0.5,0.5)
r

(0.32333333333333314,
 0.25,
 'optimal',
 'optimal',
 array([ 0.        ,  0.        , -0.82333333]),
 array([0. , 0. , 0.5]))

In [6]:
r = f(xs,zs,ys_pos,ys_neg,0.5,0.5,r[-2],r[-1],0.5,0.5)
r

(0.22904761904761883,
 0.25,
 'optimal',
 'optimal',
 array([-0.75757576,  0.        , -0.06575758]),
 array([0.54935065, 0.        , 0.04493506]))

---

In [7]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([1,1,1]),np.array([1,1,1]),1,1)
r

(0.020000000000000018,
 0.010000000000000002,
 'optimal',
 'optimal',
 array([-0.07333333,  0.        ,  0.        ]),
 array([0.05333333, 0.        , 0.        ]))

In [8]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([1,1,1]),np.array([1,1,1]),1,1)
r

(0.020000000000000018,
 0.010000000000000002,
 'optimal',
 'optimal',
 array([-0.07333333,  0.        ,  0.        ]),
 array([0.05333333, 0.        , 0.        ]))

---

In [9]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y,np.array([1,1,1]),np.array([1,1,1]),1,1)
            arr = [*r[-2],*r[-1]]
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; l \u2208 [{:.2f},{:.2f}]".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),min(arr),max(arr)))

0.1 0.1 :fx= +0.0200, +0.0100, Δfx: 0.0100; l ∈ [-0.07,0.05]
0.1 0.2 :fx= +0.0200, +0.0200, Δfx: 0.0000; l ∈ [-0.10,0.07]
0.1 0.3 :fx= +0.0067, +0.0300, Δfx: 0.0233; l ∈ [-0.15,0.14]
0.1 0.4 :fx= +0.0067, +0.0400, Δfx: 0.0333; l ∈ [-14.47,14.70]
0.1 0.5 :fx= +0.0167, +0.0500, Δfx: 0.0333; l ∈ [-15.37,15.69]
0.1 0.6 :fx= +0.0467, +0.0600, Δfx: 0.0133; l ∈ [-16.27,16.68]
0.1 0.7 :fx= +0.0900, +0.0700, Δfx: 0.0200; l ∈ [-17.30,17.82]
0.1 0.8 :fx= +0.1067, +0.0800, Δfx: 0.0267; l ∈ [-18.73,19.40]
0.1 0.9 :fx= +0.1233, +0.0900, Δfx: 0.0333; l ∈ [-20.30,21.12]
0.2 0.1 :fx= +0.0200, +0.0200, Δfx: 0.0000; l ∈ [-0.10,0.07]
0.2 0.2 :fx= +0.0400, +0.0400, Δfx: 0.0000; l ∈ [-0.12,0.08]
0.2 0.3 :fx= +0.0569, +0.0600, Δfx: 0.0031; l ∈ [-0.23,0.18]
0.2 0.4 :fx= +0.0739, +0.0800, Δfx: 0.0061; l ∈ [-0.35,0.28]
0.2 0.5 :fx= +0.0908, +0.1000, Δfx: 0.0092; l ∈ [-0.46,0.38]
0.2 0.6 :fx= +0.1051, +0.1200, Δfx: 0.0149; l ∈ [-0.57,0.47]
0.2 0.7 :fx= +0.1175, +0.1400, Δfx: 0.0225; l ∈ [-0.68,0.57]
0.2 0.8 :fx=

In [10]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y,np.array([0,0,0]),np.array([0,0,0]),x,y)
            arr = [*r[-2],*r[-1]]
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; l \u2208 [{:.2f},{:.2f}]".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),min(arr),max(arr)))

0.1 0.1 :fx= +0.0200, +0.0100, Δfx: 0.0100; l ∈ [-0.07,0.05]
0.1 0.2 :fx= +0.0200, +0.0200, Δfx: 0.0000; l ∈ [-0.10,0.07]
0.1 0.3 :fx= +0.0067, +0.0300, Δfx: 0.0233; l ∈ [-0.15,0.14]
0.1 0.4 :fx= +0.0067, +0.0400, Δfx: 0.0333; l ∈ [-14.47,14.70]
0.1 0.5 :fx= +0.0167, +0.0500, Δfx: 0.0333; l ∈ [-15.37,15.69]
0.1 0.6 :fx= +0.0467, +0.0600, Δfx: 0.0133; l ∈ [-16.27,16.68]
0.1 0.7 :fx= +0.0900, +0.0700, Δfx: 0.0200; l ∈ [-17.30,17.82]
0.1 0.8 :fx= +0.1067, +0.0800, Δfx: 0.0267; l ∈ [-18.73,19.40]
0.1 0.9 :fx= +0.1233, +0.0900, Δfx: 0.0333; l ∈ [-20.30,21.12]
0.2 0.1 :fx= +0.0200, +0.0200, Δfx: 0.0000; l ∈ [-0.10,0.07]
0.2 0.2 :fx= +0.0400, +0.0400, Δfx: 0.0000; l ∈ [-0.12,0.08]
0.2 0.3 :fx= +0.0569, +0.0600, Δfx: 0.0031; l ∈ [-0.23,0.18]
0.2 0.4 :fx= +0.0739, +0.0800, Δfx: 0.0061; l ∈ [-0.35,0.28]
0.2 0.5 :fx= +0.0908, +0.1000, Δfx: 0.0092; l ∈ [-0.46,0.38]
0.2 0.6 :fx= +0.1051, +0.1200, Δfx: 0.0149; l ∈ [-0.57,0.47]
0.2 0.7 :fx= +0.1175, +0.1400, Δfx: 0.0225; l ∈ [-0.68,0.57]
0.2 0.8 :fx=